In [5]:
import random

class Course:
    def __init__(self, code, title, credit_hours, prerequisites=None):
        self.code = code
        self.title = title
        self.credit_hours = credit_hours
        self.prerequisites = set() if prerequisites is None else set(prerequisites)

def generate_study_plan(curriculum, current_semester, study_plan):
    curriculum_remaining = curriculum.copy()
    semester_courses = []

    for course_code, course in curriculum[current_semester].items():
        prerequisites = course.prerequisites
        if prerequisites is None or set(prerequisites).issubset(set([c.code for c in study_plan[current_semester - 1]])):
            semester_courses.append(course_code)

    study_plan[current_semester - 1] = semester_courses
    curriculum_remaining.pop(current_semester)

    if current_semester < total_semesters:
        return generate_study_plan(curriculum_remaining, current_semester + 1, study_plan)
    else:
        return study_plan



def calculate_fitness(study_plan, emotional_state):
    fitness = 0
    passed_courses = set()
    for semester_courses in study_plan:
        for course_code in semester_courses:
            passed_courses.add(course_code)
            fitness += emotional_state.get(course_code, 0)
    return fitness


def selection(population, fitness_function):
    sorted_population = sorted(population, key=fitness_function, reverse=True)
    fitness_sum = sum(fitness_function(chromosome) for chromosome in population)

    probabilities = [fitness_function(chromosome) / fitness_sum for chromosome in population]
    selected_indices = random.choices(range(len(population)), probabilities, k=len(population))

    return [population[i] for i in selected_indices]

def crossover(parent1, parent2):
    crossover_point = random.randint(1, len(parent1) - 1)
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2

def mutation(study_plan):
    mutated_study_plan = study_plan.copy()

    for i in range(len(study_plan)):
        if random.random() < mutation_rate:
            semester_courses = mutated_study_plan[i]
            if semester_courses:
                index = random.randint(0, len(semester_courses) - 1)
                mutated_study_plan[i][index] = random.choice(list(curriculum[i + 1].keys()))

    return mutated_study_plan

curriculum = {
    1: {
        "sc1201": Course("sc1201", "Applied Physics", "3+0", "FRESHMAN STANDING"),
        "sc1001": Course("sc1001", "Calculus & Analytic Geometry", "3+0", "FRESHMAN STANDING"),
        "hu1002": Course("hu1002", "English Composition & Comprehension", "3+0", "FRESHMAN STANDING"),
        "cs1501": Course("cs1501", "Introduction to Information and Communication Technologies", "1+1", "FRESHMAN STANDING"),
        "cs1001": Course("cs1001", "Programming Fundamentals", "3+1", "FRESHMAN STANDING")
    },
    2: {
        "hu1003": Course("hu1003", "Communication & Presentation Skills", "3+0", "FRESHMAN STANDING"),
        "cs1502": Course("cs1502", "Digital Logic and Design", "3+1", "FRESHMAN STANDING"),
        "hu1101": Course("hu1101", "Islamic Studies", "2+0", "FRESHMAN STANDING"),
        "sc1002": Course("sc1002", "Multivariate Calculus", "2+0", "sc1001"),
        "hu1102": Course("hu1102", "Pakistan Studies", "2+0", "FRESHMAN STANDING"),
        "cs2301": Course("cs2301", "Discrete Structures", "2", "FRESHMAN STANDING"),
        "cs1002": Course("cs1002", "Programming Techniques", "1+1", "cs1001")
    },
    3: {
        "cs2503": Course("cs2503", "Computer Organization & Assembly Language", "3+1", "SPM"),
        "cs2003": Course("cs2003", "Data Structure and Algorithms", "3+1", "cs1001"),
        "sc2003": Course("sc2003", "Differential Equations", "3+0", "sc1001"),
        "hu1103": Course("hu1103", "Pakistan Studies", "2", "FRESHMAN STANDING"),
        "cs2302": Course("cs2302", "Discrete Structures-II", "3+0", "FRESHMAN STANDING"),
        "cs2004": Course("cs2004", "Object Oriented Programming", "3+1", "cs1001")
    },
    4: {
        "cs2201": Course("cs2201", "Introduction to Database Systems", "3+1", "cs2003"),
        "cs2504": Course("cs2504", "Operating Systems", "3+1", "CS2003"),
        "sc2004": Course("sc2004", "Probability and Statistics", "3+0", "SOPHOMORE STANDING"),
        "cs2101": Course("cs2101", "Software Engineering", "3+0", "SOPHOMORE STANDING")
    },
    5: {
        "cs4303": Course("cs4303", "Artificial Intelligence", "3+1", "cs2003"),
        "cs3005": Course("cs3005", "Design & Analysis of Algorithms", "3+0", "cs2003"),
        "cs3202": Course("cs3202", "Web Engineering", "3+1", "cs2003"),
        "cs3002": Course("cs3002", "Computer Networks", "3+0", "cs2003"),
        "cs3003": Course("cs3003", "Software Project Management", "3+0", "cs2101")
    },
    6: {
        "cs3701": Course("cs3701", "Computer Graphics", "3+1", "cs2004"),
        "cs4102": Course("cs4102", "Software Project-I", "3+1", "cs2004"),
        "cs3702": Course("cs3702", "Theory of Automata", "3+0", "cs2004"),
        "cs4001": Course("cs4001", "Digital Image Processing", "3+0", "cs2004"),
        "cs4602": Course("cs4602", "Data Warehousing & Data Mining", "3+0", "cs2201")
    },
    7: {
        "cs4701": Course("cs4701", "Introduction to Machine Learning", "3+1", "CS3005"),
        "cs4802": Course("cs4802", "Software Project-II", "3+1", "cs4102"),
        "cs4201": Course("cs4201", "Advanced Database Systems", "3+0", "cs2201"),
        "cs4302": Course("cs4302", "Software Quality Assurance", "3+0", "cs3003"),
        "cs4503": Course("cs4503", "Compiler Construction", "3+1", "cs3702"),
        "cs4004": Course("cs4004", "Parallel Computing", "3+0", "cs3002")
    },
    8: {
        "cs4801": Course("cs4801", "Final Year Project", "0+4", "SENIOR STANDING"),
        "cs4902": Course("cs4902", "Software Project-III", "3+1", "cs4802"),
        "cs4502": Course("cs4502", "Advanced Operating Systems", "3+1", "cs2504"),
        "cs4603": Course("cs4603", "Human Computer Interaction", "3+0", "JUNIOR STANDING"),
        "cs4005": Course("cs4005", "Wireless Networks", "3+0", "cs3002")
    }
}

emotional_states = {
"happy": 1.2,
"neutral": 1.0,
"stressed": 0.8,
"overwhelmed": 0.5
}


mutation_rate = 0.1

current_semester = int(input("Enter your current semester: "))
emotional_state = input("Enter your emotional state (happy, neutral, stressed, overwhelmed): ")

population_size = 10
total_semesters = 8
#current_semester = 1
generations = 100


population = []
for _ in range(population_size):
    study_plan = generate_study_plan(curriculum, current_semester, [[] for _ in range(current_semester, total_semesters + 1)])
    population.append(study_plan)


for generation in range(generations):
    fitness_scores = []
    for study_plan in population:
        fitness = calculate_fitness(study_plan, emotional_state)
        fitness_scores.append(fitness)

best_study_plan = population[0] # Assuming the first study plan has the highest fitness
best_fitness = calculate_fitness(best_study_plan, emotional_state)

for study_plan in population[1:]:
    fitness = calculate_fitness(study_plan, emotional_state)
    if fitness > best_fitness:
        best_study_plan = study_plan
        best_fitness = fitness

print("Best Study Plan:")
for semester, courses in enumerate(best_study_plan, start=1):
    print(f"Semester {semester}: {courses}")




IndexError: list index out of range